In [1]:
import obspy
import pandas as pd
from obspy.clients.fdsn import Client
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
import logging
import requests
import seisbench
import seisbench.models as sbm
import dask
import json
import alaska_utils
import postprocess
import eqt_utils

# Progress bar for dask operations:
from dask.diagnostics import ProgressBar
pbar = dask.diagnostics.ProgressBar()
pbar.register()

import warnings
warnings.filterwarnings('ignore')

### This notebook allows you to apply EQTransformer (by way of Seisbench) to continuous seismic data from the AACSE network, and retrieve picks for the specified time period. The waveforms can also be filtered or denoised by DeepDenoiser (by way of Seisbench).

#### Choose window length (s), window overlap (s), and start and end times

In [2]:
waveform_length = 60
waveform_overlap = 0
starttime = datetime.datetime(2019,5,27,9,52,30)
endtime = datetime.datetime(2019,5,27,9,55,30)

#### Load list of stations to run the workflow on, specify by network and maximum station elevation if desired

In [3]:
# Pre=saved station list in pandas dataframe format
dfS = pd.read_parquet('https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_stations.parquet')
# dfS = alaska_utils.station_list(dfS,starttime,endtime,elevation=False,network=False)

#### Specify whether to apply any filtering to the waveforms before phase picking

This choice is made with the variable filt_type.

filt_type = 0 for raw waveforms\
filt_type = 1 for bandpass filtered waveforms\
filt_type = 2 to apply DeepDenoiser to the waveforms

NOTE: if specifying bandpass filter, must also specify f1 and f2 for bandpass limits

In [4]:
filt_type = 2
f1 = False
f2 = False

### Run the workflow, and output:
#### (1) pandas dataframe of all pick metadata
#### (2) list of pick dictionaries formatted for use with GAMMA

In [5]:
%%time

output_picks_filtered,output_gamma = eqt_utils.ml_pick(dfS,starttime,endtime,waveform_length,waveform_overlap,filt_type,f1=f1,f2=f2)

[########################################] | 100% Completed | 49.3s
Status: denoising data using DeepDenoiser
Status: applying EQTransformer


2022-02-16 19:55:10,232 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-16 19:55:19,571 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.


CPU times: user 59.4 s, sys: 757 ms, total: 1min
Wall time: 1min 27s


#### Below is an example of the output pick dataframe. You can see that both the raw and denoised SNR of the pick is saved for all channels, as well as the raw and denoised amplitude of the signal (for use in magnitude estimation with GAMMA), and the probability of the pick from EQTransformer.

In [6]:
output_picks_filtered.head()

,id,network,station,channel,phase,timestamp,prob,raw_amp,den_amp,z_raw_snr,z_den_snr,n_raw_snr,n_den_snr,e_raw_snr,e_den_snr
0,XO.EP15..HH,XO,EP15,HH,P,2019-05-27T09:52:44.390,0.703371,540944.0,5.109854e+05,1066.908243,1504.774816,1085.470799,29300.145520,570.053574,11851.051777
1,XO.EP15..HH,XO,EP15,HH,S,2019-05-27T09:53:01.930,0.342681,1570463.0,1.351395e+06,44.920476,12.229282,160.105093,22.075242,193.399981,14.533355
2,XO.ET17..HH,XO,ET17,HH,P,2019-05-27T09:52:53.160,0.378271,161573.0,1.238372e+05,446.658960,9507.911471,153.959334,5453.627586,69.863421,2648.617765
3,XO.ET19..HH,XO,ET19,HH,P,2019-05-27T09:52:48.249,0.458545,90903.0,4.436333e+04,164.076697,3315.300102,56.639121,56629.467835,53.024173,474.351397
4,XO.ET19..HH,XO,ET19,HH,S,2019-05-27T09:53:09.539,0.444751,382374.0,1.795152e+05,8.622256,1.687317,16.175838,12.142870,22.626180,11.097535


### Save to parquet

In [7]:
output_picks_filtered.to_parquet('picks_20190527_095230.parquet', version='2.6')